## Import Dataset and EDA 

In [1]:
#Modules
import pandas as pd
import numpy as np
from ast import literal_eval
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.test.utils 
import tqdm
%matplotlib inline
import matplotlib.pyplot as plt
import pickle

In [2]:
kremlin = pd.read_excel('data/putin_corpus.xlsx')
kremlin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33189 entries, 0 to 33188
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          33189 non-null  object
 1   URL           33189 non-null  object
 2   description   30316 non-null  object
 3   introduction  30316 non-null  object
 4   id            33189 non-null  int64 
 5   speaker       33189 non-null  object
 6   p             33189 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.8+ MB


In [3]:
kremlin.rename(columns = {'p':'text'}, inplace = True)

In [5]:
print(kremlin.iloc[2666]["text"])

Ежели эти люди, о которых Вы сейчас сказали, пройдя через это, придут к этому, мы будем только аплодировать. Понимаете, некоторые, на мой взгляд, может быть, вы на меня сильно обидитесь, но пиарятся в тюрьмах. Вот это страшно.


In [10]:
print(kremlin.iloc[140]["description"])

Владимир Путин поздравил Патриарха Кирилла с Днём тезоименитства, праздником Вознесения и Днём славянской письменности. Глава государства преподнёс в подарок предстоятелю Русской православной церкви икону «Вознесение».Затем Владимир Путин и Патриарх Московский и всея Руси Кирилл встретились с архиепископом Афинским и всея Эллады Иеронимом II, совершающим визит в Россию.


In [ ]:
#Identify Noise with Regular Expression
import re 
RE_SUSPICIOUS = re.compile(r'[Äô&#<>{}\[\]\\]')

def impurity(text, min_len=10): 
    """returns the share of suspicious characters in a text""" 
    if text == None or len(text) < min_len: 
        return 0 
    else: 
        return len(RE_SUSPICIOUS.findall(text))/len(text)


In [ ]:
# add new column to data frame
kremlin['impurity'] = kremlin['text'].apply(impurity, min_len=10)

In [ ]:
kremlin['impurity'].sort_values().value_counts()

In [ ]:
# get the top 3 records 
#critical part is just []...I will leave as it is for the moment
kremlin[['text', 'impurity']].sort_values(by='impurity', ascending=False).head(3)

In [ ]:
def clean(text):
     # everything non-alpahnumeric with a space
     text = re.sub(r'\W+',' ', text)
     # Two or more dots with one
     text = re.sub(r'\.{2,}', ' ', text)
     # sequences of white spaces
     text = re.sub(r'\s+', ' ', text)
     #convert text to lowercase
     text = text.lower()
     return text.strip()

In [ ]:
kremlin['text'] = kremlin['text'].map(clean)

In [ ]:
# get the top 3 records 
#critical part is just []...I will leave as it is for the moment
kremlin[['text', 'impurity']].sort_values(by='impurity', ascending=False).head(3)

In [ ]:
kremlin.drop(columns=['impurity'], inplace=True)

In [ ]:
print(repr(kremlin.iloc[14288]["text"]))

In [ ]:
print(kremlin.iloc[29683]["text"])

## Text Preprocessing

In [ ]:
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.ru.stop_words import STOP_WORDS

In [ ]:
nlp_spacy = spacy.load('ru_core_news_sm', exclude=["ner"])
nlp_spacy.disable_pipe("parser")
nlp_spacy.enable_pipe("senter")

In [ ]:
#let’s build phrase models that we can clean and use to create the corpus for our analysis
bigram = gensim.models.Phrases(kremlin['text'], min_count = 5,
                               threshold = 100)

In [ ]:
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [ ]:
#Stopwords
from spacy.lang.ru import stop_words
nlp_spacy.Defaults.stop_words |= { 'дорогой', 'день', 'коллега', 'добрый', 'два', 'раз', 'сегодня', 'спасибо', 'уважаемый', 'уважаемыe'}
stop_words = stop_words.STOP_WORDS

In [ ]:
#remove stopwords
def remove_stopwords(texts):
    return[[word for word in simple_preprocess(str(doc)) if word not in stop_words]
            for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
#Turn words into lemmas
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp_spacy(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
data_words_nostops = remove_stopwords(kremlin['text'])

In [ ]:
data_words_bigram = make_bigrams(data_words_nostops)

In [ ]:
data_lemma = lemmatization(data_words_bigram, 
                           allowed_postags=['NOUN', 'ADJ', 'VERB'])

In [ ]:
lemma = pd.DataFrame({'lemma': data_lemma})

In [ ]:
lemma

In [ ]:
lemma.to_pickle('data/lemma.pkl')

In [ ]:
read_lemma = pd.read_pickle('data/lemma.pkl')

In [ ]:
read_lemma

In [10]:
#if read_lemma works, then it's necessary to create again a list of a list
read_lemma = pd.read_pickle('data/lemma.pkl')
data_lemma = read_lemma['lemma'].tolist()

In [86]:
#dictionary
id2word = corpora.Dictionary(data_lemma)
#corpus
texts = data_lemma
#term document matrix
corpus = [id2word.doc2bow(text) for text in texts]

In [118]:
[[(id2word[i], freq) for i, freq in doc] for doc in corpus[:2]]

[[('гражданин', 1), ('дорогой', 1), ('друг', 1), ('уважаемые', 1)],
 [('гражданин', 1),
  ('внешний', 1),
  ('внутренний', 1),
  ('вступление', 1),
  ('гарант', 1),
  ('глава', 1),
  ('государство', 1),
  ('должность', 1),
  ('жизнь', 1),
  ('избрать', 1),
  ('история', 1),
  ('конституция', 1),
  ('направление', 1),
  ('начало', 1),
  ('новый', 1),
  ('определять', 1),
  ('основный', 1),
  ('политика', 1),
  ('право', 1),
  ('президент', 2),
  ('свобода', 1),
  ('человек', 1),
  ('этап', 1),
  ('являться', 1)]]

In [87]:
print('Number of unique tokens: %d' % len(id2word))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 26650
Number of documents: 33189


## Training the Models

### num_of_topics=10 

In [88]:
#TRAINING THE MODEL (num_of_topics=10)
num_topics = 10
chunksize = 100
passes = 10 #controls how often we train the model on the entire corpus.
iterations = 200
eval_every = None  # Don't evaluate model perplexity, takes too much time.

In [89]:
lda_model_10 = gensim.models.LdaMulticore(corpus = corpus,
                                       id2word = id2word,
                                       num_topics = num_topics,
                                       random_state = 42,
                                       chunksize = chunksize,
                                       passes = passes,
                                       per_word_topics=True,
                                       minimum_probability = 0)

In [90]:
pprint(lda_model_10.print_topics())
doc_lda = lda_model_10[corpus]

[(0,
  '0.027*"страна" + 0.020*"экономический" + 0.017*"международный" + '
  '0.016*"отношение" + 0.015*"форум" + 0.015*"российский" + 0.012*"регион" + '
  '0.011*"развитие" + 0.011*"республика" + 0.010*"связь"'),
 (1,
  '0.029*"центр" + 0.017*"медицинский" + 0.015*"новый" + 0.013*"школа" + '
  '0.012*"образование" + 0.011*"область" + 0.011*"наука" + 0.011*"современный" '
  '+ 0.011*"товарищ" + 0.011*"научный"'),
 (2,
  '0.033*"друг" + 0.026*"уважаемые" + 0.025*"коллега" + 0.023*"успех" + '
  '0.021*"дорогой" + 0.020*"президент" + 0.019*"поблагодарить" + 0.016*"слово" '
  '+ 0.015*"праздник" + 0.015*"поздравлять"'),
 (3,
  '0.050*"внимание" + 0.050*"большой" + 0.043*"ребёнок" + 0.040*"город" + '
  '0.037*"семья" + 0.026*"благодарить" + 0.025*"здоровье" + 0.015*"игра" + '
  '0.013*"дом" + 0.012*"обратить"'),
 (4,
  '0.020*"народ" + 0.017*"страна" + 0.015*"история" + 0.014*"человек" + '
  '0.012*"великий" + 0.012*"искусственный" + 0.011*"война" + 0.011*"сила" + '
  '0.009*"жизнь" + 0.009

In [91]:
#save model in pickle format to working directory
lda_model_10.save("models/lda_model_10.pkl")

In [92]:
#load model back into your workspace from working directory
lda_10 = gensim.models.LdaModel.load("models/lda_model_10.pkl")

In [93]:
#u_mass
coherence_model_lda_10 = CoherenceModel(model=lda_model_10, corpus=corpus, coherence='u_mass')
coherence_lda_10 = coherence_model_lda_10.get_coherence()
print('\nCoherence Score: ', coherence_lda_10)


Coherence Score:  -3.726749524769879


In [119]:
#c_v
coherence_model_lda_10 = CoherenceModel(model=lda_model_10, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda_10.get_coherence()
print('nCoherence Score: ', coherence_lda)

/opt/anaconda3/envs/nlp/lib/python3.9/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
/opt/anaconda3/envs/nlp/lib/python3.9/site-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


nCoherence Score:  nan


### num_of_topics = 30

In [94]:
#TRAINING THE MODEL (num_of_topics=30)
num_topics = 30
chunksize = 100
passes = 10 #controls how often we train the model on the entire corpus.
iterations = 200
eval_every = None  # Don't evaluate model perplexity, takes too much time.

In [95]:
lda_model_30 = gensim.models.LdaMulticore(corpus = corpus,
                                       id2word = id2word,
                                       num_topics = num_topics,
                                       random_state = 42,
                                       chunksize = chunksize,
                                       passes = passes,
                                       per_word_topics=True,
                                       minimum_probability = 0)

In [96]:
pprint(lda_model_30.print_topics())
doc_lda = lda_model_30[corpus]

[(16,
  '0.067*"успех" + 0.052*"поздравлять" + 0.043*"праздник" + 0.043*"желать" + '
  '0.033*"добрый" + 0.032*"новый" + 0.030*"здоровье" + 0.030*"чемпионат" + '
  '0.029*"восточный" + 0.025*"работа"'),
 (2,
  '0.049*"спорт" + 0.042*"команда" + 0.039*"выступление" + 0.038*"школа" + '
  '0.034*"игра" + 0.028*"язык" + 0.025*"спортивный" + 0.022*"единство" + '
  '0.021*"относиться" + 0.021*"массовый"'),
 (20,
  '0.037*"компания" + 0.036*"экономика" + 0.033*"рынок" + 0.026*"рост" + '
  '0.026*"бизнес" + 0.020*"мировой" + 0.017*"отрасль" + 0.017*"процент" + '
  '0.017*"малый" + 0.016*"цена"'),
 (12,
  '0.256*"год" + 0.053*"тысяча" + 0.044*"процент" + 0.040*"миллиард" + '
  '0.032*"последний" + 0.032*"прошлый" + 0.028*"доллар" + 0.023*"миллион" + '
  '0.021*"рубль" + 0.015*"принять"'),
 (9,
  '0.060*"человек" + 0.047*"говорить" + 0.039*"знать" + 0.032*"сделать" + '
  '0.029*"первый" + 0.029*"сказать" + 0.027*"работать" + 0.026*"думать" + '
  '0.025*"делать" + 0.023*"хотеть"'),
 (17,
  '0.061

In [108]:
#save model in pickle format to working directory
lda_model_30.save("models/lda_model_30.pkl")

In [109]:
#load model back into your workspace from working directory
lda_30 = gensim.models.LdaModel.load("models/lda_model_30.pkl")

In [99]:
coherence_model_lda_30 = CoherenceModel(model=lda_model_30, corpus=corpus, coherence='u_mass')
coherence_lda_30 = coherence_model_lda_30.get_coherence()
print('\nCoherence Score: ', coherence_lda_30)


Coherence Score:  -5.105336650761064


### num_of_topics = 50

In [105]:
#TRAINING THE MODEL (num_of_topics=50)
num_topics = 50
chunksize = 100
passes = 10 #controls how often we train the model on the entire corpus.
iterations = 200
eval_every = None  # Don't evaluate model perplexity, takes too much time.

In [106]:
lda_model_50 = gensim.models.LdaMulticore(corpus = corpus,
                                       id2word = id2word,
                                       num_topics = num_topics,
                                       random_state = 42,
                                       chunksize = chunksize,
                                       passes = passes,
                                       per_word_topics=True,
                                       minimum_probability = 0)

In [110]:
pprint(lda_model_50.print_topics())
doc_lda = lda_model_50[corpus]

[(0,
  '0.057*"интересный" + 0.050*"образ" + 0.042*"помощь" + 0.038*"оказывать" + '
  '0.036*"увидеть" + 0.035*"взгляд" + 0.031*"церковь" + 0.029*"собраться" + '
  '0.028*"передать" + 0.023*"представитель"'),
 (11,
  '0.144*"слово" + 0.067*"сказать" + 0.057*"благодарность" + 0.052*"контакт" + '
  '0.047*"дать" + 0.046*"присутствовать" + 0.043*"зал" + 0.038*"уважаемый" + '
  '0.030*"террорист" + 0.026*"отдельный"'),
 (2,
  '0.070*"спорт" + 0.057*"вид" + 0.054*"следующий" + 0.049*"посмотреть" + '
  '0.046*"вызов" + 0.036*"выступление" + 0.033*"спортивный" + 0.027*"целом" + '
  '0.023*"увеличиться" + 0.018*"приходить"'),
 (15,
  '0.046*"создавать" + 0.043*"задача" + 0.041*"решать" + 0.031*"информация" + '
  '0.030*"момент" + 0.027*"перспектива" + 0.027*"угроза" + 0.025*"океан" + '
  '0.022*"содействие" + 0.021*"народный"'),
 (22,
  '0.061*"общий" + 0.061*"страна" + 0.049*"регион" + 0.043*"усилие" + '
  '0.043*"развитие" + 0.036*"сотрудничество" + 0.032*"диалог" + 0.029*"мир" + '
  '0.029*

In [111]:
#save model in pickle format to working directory
lda_model_50.save("models/lda_model_50.pkl")

In [112]:
#load model back into your workspace from working directory
lda_50 = gensim.models.LdaModel.load("models/lda_model_50.pkl")

In [113]:
coherence_model_lda_50 = CoherenceModel(model=lda_model_50, corpus=corpus, coherence='u_mass')
coherence_lda_50 = coherence_model_lda_50.get_coherence()
print('\nCoherence Score: ', coherence_lda_50)


Coherence Score:  -6.037390920317644


### visualize topic models

In [7]:
from imp import reload
import pyLDAvis
import pyLDAvis.gensim_models

/var/folders/mm/gw92vpys6rggjz_4z42s244w0000gn/T/ipykernel_1468/2769710462.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [8]:
pyLDAvis.enable_notebook()

In [101]:
from imp import reload
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
pyLDAvis.gensim_models.prepare(lda_10, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.189445  0.002433       1        1  19.212432
6     -0.178833 -0.081056       2        1  15.035201
5     -0.144282  0.100496       3        1  12.346823
4     -0.014514 -0.102081       4        1  11.354027
0     -0.100676 -0.042865       5        1  10.787554
8      0.019482  0.032438       6        1   8.345199
2      0.183223 -0.090148       7        1   7.197680
1      0.055430 -0.208685       8        1   6.412697
9      0.016380  0.342739       9        1   5.691100
3      0.353235  0.046730      10        1   3.617287, topic_info=              Term         Freq        Total Category  logprob  loglift
31             год  6560.000000  6560.000000  Default  30.0000  30.0000
24         человек  4865.000000  4865.000000  Default  29.0000  29.0000
584         вопрос  2851.000000  2851.000000  Default  28.0000  28.0000
190        большой  1850.000000  1850.000000  Default  27.0000  27.0000
87       экономика  2280.000000  2280.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
168   благополучие   173.031488   263.083059  Topic10  -4.8758   2.9004
1582         среда   150.060007   288.014825  Topic10  -5.0182   2.6675
177      поддержка   242.011971  1192.540717  Topic10  -4.5403   1.7246
1016        особый   156.134679   524.388399  Topic10  -4.9785   2.1079
755     находиться   127.599603   733.604857  Topic10  -5.1804   1.5704

[561 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1610      6  0.996466       атомный
2214      3  0.993419   африканский
2881      2  0.995474        баланс
27        1  0.812869  безопасность
27        5  0.186404  безопасность
...     ...       ...           ...
26        3  0.109537      являться
26        4  0.118859      являться
26        5  0.309966      являться
26        8  0.000777      являться
2830      4  0.997254          язык

[885 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 7, 6, 5, 1, 9, 3, 2, 10, 4])

In [114]:
from imp import reload
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
pyLDAvis.gensim_models.prepare(lda_30, corpus, id2word)

/opt/anaconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
9      0.108500 -0.068872       1        1  6.164145
28     0.221586 -0.005156       2        1  5.795531
10     0.173198  0.104338       3        1  5.774280
27    -0.032999 -0.275908       4        1  5.734266
1      0.111511 -0.263905       5        1  5.330388
13     0.222373 -0.035968       6        1  5.324582
19     0.152564  0.100568       7        1  5.124354
4      0.100759 -0.081353       8        1  4.537298
20     0.091386  0.122651       9        1  4.130378
12    -0.034510  0.064748      10        1  3.778993
15     0.199885  0.077274      11        1  3.423332
29     0.030050  0.104777      12        1  3.185631
25     0.055286  0.115439      13        1  3.147923
21     0.013169  0.063978      14        1  3.035496
14    -0.017113 -0.017230      15        1  2.954344
22    -0.059876 -0.068538      16        1  2.922821
26    -0.003680 -0.034644      17        1  2.847257
18    -0.108237 -0.005045      18        1  2.557319
23    -0.004051 -0.015232      19        1  2.405010
7     -0.075642  0.051560      20        1  2.366027
5     -0.092461 -0.016097      21        1  2.246660
11    -0.084031  0.026595      22        1  2.231947
16    -0.107940 -0.048462      23        1  2.212980
6      0.010131  0.074974      24        1  2.104904
17    -0.077430 -0.169563      25        1  2.075600
24    -0.161542  0.044220      26        1  1.897597
2     -0.095947 -0.095184      27        1  1.786831
8     -0.196399  0.102432      28        1  1.660228
3     -0.156967  0.091399      29        1  1.636221
0     -0.181575  0.056203      30        1  1.607655, topic_info=               Term         Freq        Total Category  logprob  loglift
31              год  8039.000000  8039.000000  Default  30.0000  30.0000
748         сказать  3572.000000  3572.000000  Default  29.0000  29.0000
22        президент  2526.000000  2526.000000  Default  28.0000  28.0000
555        говорить  3771.000000  3771.000000  Default  27.0000  27.0000
24          человек  4762.000000  4762.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
204   ответственный   136.412724   314.657641  Topic30  -4.3026   3.2946
1282  преувеличение    93.011251   146.748375  Topic30  -4.6856   3.6744
464        доказать    71.786801   104.836726  Topic30  -4.9446   3.7517
182           самые    93.711276   523.968813  Topic30  -4.6781   2.4092
1500     физический    73.838249   129.777389  Topic30  -4.9164   3.5664

[1585 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
3464     30  0.989564       xxi
2077      4  0.124634   авиация
2077     15  0.117303   авиация
2077     21  0.146629   авиация
2077     22  0.564521   авиация
...     ...       ...       ...
26       27  0.014672  являться
2830     27  0.996613      язык
2597      9  0.992659    январь
992       4  0.426238     яркий
992      15  0.572466     яркий

[3084 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 29, 11, 28, 2, 14, 20, 5, 21, 13, 16, 30, 26, 22, 15, 23, 27, 19, 24, 8, 6, 12, 17, 7, 18, 25, 3, 9, 4, 1])

In [115]:
pyLDAvis.gensim_models.prepare(lda_50, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
23    -0.228320  0.061471       1        1  4.021883
27     0.031184 -0.248596       2        1  3.794507
40    -0.226304  0.094787       3        1  3.484528
28    -0.120543  0.169194       4        1  3.319987
19    -0.079531  0.080885       5        1  3.192612
48    -0.185736 -0.011947       6        1  3.184895
44    -0.173963 -0.004410       7        1  3.132513
22    -0.192736 -0.102243       8        1  2.985711
7     -0.117865 -0.084229       9        1  2.820939
35     0.014636 -0.098874      10        1  2.800461
21     0.039465  0.105685      11        1  2.769133
18    -0.064745 -0.265086      12        1  2.630765
20    -0.115256  0.039895      13        1  2.507634
31    -0.043585 -0.048178      14        1  2.400720
30    -0.108185  0.140078      15        1  2.297923
9      0.068858 -0.124273      16        1  2.247484
6     -0.138333 -0.032315      17        1  2.099484
14    -0.071933  0.012068      18        1  1.962670
38     0.048559 -0.040742      19        1  1.906262
39     0.048217  0.053282      20        1  1.890477
15    -0.094718  0.049690      21        1  1.883562
29    -0.049361 -0.074136      22        1  1.843449
49     0.022974  0.096251      23        1  1.831918
10     0.055252  0.080557      24        1  1.828315
8     -0.005846  0.052487      25        1  1.745528
16     0.018625 -0.086513      26        1  1.734590
4     -0.009411 -0.069774      27        1  1.725538
41     0.023673  0.047694      28        1  1.689327
26    -0.048228  0.033568      29        1  1.670667
47     0.045919  0.038918      30        1  1.668037
1      0.032405 -0.145727      31        1  1.638204
24     0.077849 -0.024987      32        1  1.585902
2      0.015532  0.031821      33        1  1.569339
17     0.048485 -0.129863      34        1  1.528706
46     0.055870  0.053053      35        1  1.517522
36     0.100806  0.025100      36        1  1.501539
37     0.045307  0.037689      37        1  1.443173
13     0.041066  0.101561      38        1  1.440295
43     0.011912 -0.032785      39        1  1.423814
5     -0.001906 -0.063838      40        1  1.366621
32     0.157662  0.036007      41        1  1.350559
11     0.134073  0.000421      42        1  1.331985
42     0.089892 -0.032950      43        1  1.326382
3      0.123225  0.074535      44        1  1.319098
0      0.104487  0.010296      45        1  1.259946
33     0.136382  0.031486      46        1  1.202828
34     0.094141  0.020053      47        1  1.151727
25     0.097731  0.096828      48        1  1.091570
45     0.168455  0.020279      49        1  0.992331
12     0.123863  0.025825      50        1  0.886946, topic_info=                 Term         Freq        Total Category  logprob  loglift
31                год  8029.000000  8029.000000  Default  30.0000  30.0000
24            человек  4913.000000  4913.000000  Default  29.0000  29.0000
141        российский  4312.000000  4312.000000  Default  28.0000  28.0000
190           большой  1966.000000  1966.000000  Default  27.0000  27.0000
147            первый  2097.000000  2097.000000  Default  26.0000  26.0000
...               ...          ...          ...      ...      ...      ...
1918       выдающийся    52.020706   301.512121  Topic50  -4.6719   2.9680
440           главный    59.865595  1119.245401  Topic50  -4.5315   1.7968
96             пройти    32.919955   798.466751  Topic50  -5.1295   1.5365
630   государственный    33.436317  1127.521696  Topic50  -5.1139   1.2070
119           считать    25.756718  1199.561941  Topic50  -5.3749   0.8841

[2501 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
3463     43  0.982076          xix
3464     43  0.987244          xxi
1940     44  0.940548    аварийный
3484     44  0.980440       август
3448     49  0.984961  авиационный
...     ...       

In [116]:
pyLDAvis.save_html(lda_10, 'graphs/lda_10.html')

AttributeError: 'LdaMulticore' object has no attribute 'to_json'